In [22]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re

In [3]:
list_topic=['pol','eco','lif','soc','int']
topic_dict={'pol':"정치",'eco':"경제",'lif':"라이프",'soc':"사회",'int':"국제"}

In [16]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20201002')

In [5]:
# 각각의 리스트 생성
topic_list=[]
date_list=[]
title_list=[]
text_list=[]

In [6]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [7]:
#크롤링 def
def get_news(date,main_url,sub_url,num,topic):
    html = urlopen(main_url+date)
    bsObject = BeautifulSoup(html, "html.parser")
    bs_page=bsObject.find(class_="paging")
    for meta in bsObject.find_all('dt'):
        link=meta.a.get('href')
        title=no_space(meta.text)
        
        title_list.append(title)
        date=date
        date_list.append(date)
        url=urlopen(sub_url+link)
        bs_obj=BeautifulSoup(url, "html.parser")
        bs_text=bs_obj.body.find(class_="tx")
        text=no_space(bs_text.text)
        text_list.append(text)
        topic_list.append(topic_dict[topic])

    if(bs_page.find(class_="end").get('href')==("list.asp?sid1="+topic+"&sid2=&page=1&sdate="+date+"&st=")):
        return bsObject
  
    if(bs_page.find(class_="end").get('href')==("list.asp?sid1="+topic+"&sid2=&page="+str(num)+"&sdate="+date+"&st=")):
        return bsObject
    else:
        #재귀 호출
        url_new="http://news.kmib.co.kr/article/list.asp?sid1="+topic+"&sid2=&page="+str(num+1)+"&sdate="
        return get_news(date,url_new,sub_url,num+1,topic)


In [17]:
#크롤링 시작
try:
    for num_topic in tqdm(range(len(list_topic)),desc="Topic_Progress"):
        main_url="http://news.kmib.co.kr/article/list.asp?sid1="+list_topic[num_topic]+"&sid2=&sdate="
        sub_url='http://news.kmib.co.kr/article/'
        for num_date in tqdm(range(len(dt_index.date)-1),desc="Day_Progress"):
            date=dt_index.date[num_date]
            date=str(date).split('-')[0]+str(date).split('-')[1]+str(date).split('-')[2]
            bs=get_news(date,main_url,sub_url,1,list_topic[num_topic])
except:
    print("에러가 발생 했습니다.")

KeyboardInterrupt: 

In [9]:
#판다스로 저장 (후에 csv파일로 저장 라인 추가)
df=pd.DataFrame({"date":date_list, "title":title_list, "topic":topic_list,"text":text_list}) 
print(df) 

Empty DataFrame
Columns: [date, title, topic, text]
Index: []


In [13]:
len(dt_index.day)

2

In [92]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait

options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver')
browser.get("https://petitions.assembly.go.kr/closed/inadequate")
browser.implicitly_wait(3)




In [93]:

browser.find_element_by_id("beginDate").send_keys('002020-01-20')
browser.find_element_by_class_name("btn_search").click()


In [94]:
x= browser.find_element_by_class_name('ListDiv')
y=x.find_element_by_tag_name("a")
print(y)
z=y.get_attribute('href')

<selenium.webdriver.remote.webelement.WebElement (session="31b177d05ff94ad4aa420c3d7fcded4d", element="4b59c7df-cab3-4eea-a440-6680c39d43fd")>


In [95]:
z=y.get_attribute('href')
z

'https://petitions.assembly.go.kr/closed/inadequate/B2A63D82BE8953EFE054A0369F40E84E'

In [96]:
browser1 = webdriver.Chrome('./chromedriver')
browser1.get(z)
browser1.implicitly_wait(3)

In [ ]:
#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul > li:nth-child(1) > div

In [83]:
x3=k.page_source
x1=BeautifulSoup(x3, 'html.parser')
x1.find(class_='petitionListDiv')

AttributeError: 'list' object has no attribute 'page_source'

In [187]:
title_list=[]
date_list=[]
result_list=[]
agree_list=[]
effect_list=[]
text_list=[]
topic_list=[]

In [188]:

driver = webdriver.Chrome('./chromedriver')
driver2 = webdriver.Chrome('./chromedriver')

driver.get("https://petitions.assembly.go.kr/closed/inadequate")
driver.find_element_by_id("beginDate").send_keys('002020-01-20')
search=driver.find_element_by_class_name("btn_search")
search.click()


    



In [209]:
next_page=driver.find_element_by_xpath('//*[@id="contentsbody"]/div/div/div[5]/div[3]/ul/li[10]/a')

while True:
    list_petition=driver.find_elements_by_css_selector('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul>li>div> a')
    for num in list_petition:
        driver2.get(num.get_attribute('href'))
        title=driver2.find_element_by_css_selector('#contentsbody > div > div > div.viewTopDiv > h4 ')
        title_list.append(title.text)
        date=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.secondline > li:nth-child(1) > div')
        date_list.append((date.text).split('\n')[0])
        result=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul:nth-child(3) > li:nth-child(2) > div')
        result_list.append(result.text)
        agree=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.secondline > li:nth-child(2) > div > div > ul > li:nth-child(1)')
        agree_list.append(agree.text)
        effect=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionTb > dl:nth-child(3) > dd')
        effect_list.append(effect.text)
        text=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionTb > dl:nth-child(4) > dd')
        text_list.append(text.text)
        topic=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul:nth-child(2) > li > div')
        topic_list.append(topic.text)
        
    try:
        next_page.click()
    except:
        break

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)


In [206]:
k=driver.find_elements_by_css_selector('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul>li>div> a' )

In [207]:
for i in k:
    print(i.get_attribute('href'))

https://petitions.assembly.go.kr/closed/inadequate/B2A63D82BE8953EFE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE91454EAAB5029FE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE8865C042B14927E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEC2BC0CE7DA59F0E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEAEF67EC69B02CAE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEC2BC0CE7BF59F0E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE61247629364827E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AF000D5096F15D8EE054A0369F40E84E


In [167]:
k=driver2.find_element_by_css_selector('#contentsbody > div > div > div.petitionViewWrap > div.petitionfield > ul.secondline > li:nth-child(1) > div')

In [173]:
(k.text).split('\n')[0]

'2020-10-15~2020-11-14'

In [212]:
text_list

['1. 국가 책임의 일원화된 돌봄서비스 제공을 위한 법 체계 마련이 필요합니다.\n돌봄은 나라의 미래를 위한 국가의 중요한 책무가 되고 있는 바, 이제 정부는 돌봄을 국가 책임 하에 보건복지부(여성가족부), 지방자치단체 사무로 일원화해야 합니다. 양질의 돌봄 서비스 제공을 위해서는 지역아동센터, 다함께돌봄, 방과후아카데미, 초등돌봄교실 등 여기저기 흩어져 운영되고 있는 돌봄이, 지방자치단체로 일원화되어 통합 운영되는 것이 바람직합니다. 그래야 예산 낭비를 줄이고 지역의 다양한 인프라를 활용해 양질의 돌봄 제공이 가능하기 때문입니다.\n한두 명의 돌봄을 위해 교사 1인과 돌봄사 1인이 투입되어야 하는 초등 저녁 돌봄교실은 아이에겐 돌봄이 아닌 수용일 뿐이며, 예산 사용상 비효율의 극치입니다. 이는 지자체가 이동서비스 제공으로 지역의 돌봄 센터에 아동을 모아 양질의 체험프로그램을 제공하는 방식으로 극복할 수 있습니다. 이를 위해서도 돌봄이 지자체로 일원화되어 통합 운영되어야 합니다.\n \n2. 진짜 아이들을 생각하는 돌봄을 위해서는 지자체 책임 운영이 필요합니다.\n아이들의 건강한 성장을 위해서는 다양한 놀이 공간과 다양한 지역사회 컨텐츠를 활용한 돌봄이 이루어질 수 있도록 해야 합니다. 동네마다 노인정과 어린이집이 존재하듯이 이제는 동네마다 돌봄 시설이 있어야 하는 시대가 되었습니다. 이는 노인 복지, 보육 복지에 국가와 지방자치단체가 관심을 갖고 투자하듯이 아동 돌봄 복지에 국가와 지방자치단체가 관심을 갖고 투자할 때 이루어질 수 있습니다. 이런 아동 돌봄 복지가 이루어지기 위해서는 돌봄 업무가 지자체 업무로 일원화되어야 합니다. 현재의 학교 돌봄은 교육 예산을 쪼개서 사용하고 있어, 학교 돌봄이 확대되면 확대될수록 아이들의 교육혜택은 줄어들 수밖에 없습니다. 모든 학생들이 더 나은 교육 혜택을 받기 위해서라도 학교 예산은 교육에 온전히 집중될 수 있도록 차후의 돌봄 확대는 국가와 지방자치단체의 복지 예산으로 이루어져야 합니다. 이를 위해서도 지자체 책임 

In [154]:
driver2.get('https://petitions.assembly.go.kr/closed/inadequate/AEAEF67EC69B02CAE054A0369F40E84E')

In [158]:
x=driver2.find_element_by_css_selector('#contentsbody > div > div > div.viewTopDiv > h4 ')
x.text

'"모자보건법 일부개정안 반대"에 대한 청원'

In [133]:
list_petition=driver.find_elements_by_css_selector('#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul >li > div')

In [135]:
for num in list_petition:
    print(num.get_attribute('href'))

https://petitions.assembly.go.kr/closed/inadequate/B2A63D82BE8953EFE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE91454EAAB5029FE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE8865C042B14927E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEC2BC0CE7DA59F0E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEAEF67EC69B02CAE054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AEC2BC0CE7BF59F0E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AE61247629364827E054A0369F40E84E
https://petitions.assembly.go.kr/closed/inadequate/AF000D5096F15D8EE054A0369F40E84E


In [132]:
for i in list_petition:
    print(i)

<selenium.webdriver.remote.webelement.WebElement (session="7309725c65e6936376eef3d8ce89ab64", element="1f8f179f-7045-4a9b-a6db-757b43ffa05f")>


In [ ]:
#contentsbody > div > div > div.petitionListWrap > div.petitionListDiv > div.ListDiv > ul > li:nth-child(1) > div > a